In [1]:
import tensorflow as tf 
import numpy as np 

2024-10-24 19:06:26.869828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 19:06:26.883934: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 19:06:26.888236: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 19:06:26.898302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 19:06:27.649256: W tensorflow/compiler/tf2

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
train_images  = train_images.reshape(-1, 28, 28, 1).astype("float32")/255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype('float32')/255.0

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)), 
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax')

])

/home/amin/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1729784188.739210   37314 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-24 19:06:28.773671: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for yo

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=2, validation_split=0.1)


Epoch 1/2
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8860 - loss: 0.3634 - val_accuracy: 0.9828 - val_loss: 0.0587
Epoch 2/2
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9824 - loss: 0.0568 - val_accuracy: 0.9887 - val_loss: 0.0409


In [6]:
#keras
model.save('mnist_saved_keras.keras')

In [7]:
#h5
model.save('mnist_saved_h5.h5')

In [8]:
#tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('mnist_saved_tflite.tflite', "wb") as f :
    f.write(tflite_model)



INFO:tensorflow:Assets written to: /tmp/tmpao5hgny1/assets


INFO:tensorflow:Assets written to: /tmp/tmpao5hgny1/assets


Saved artifact at '/tmp/tmpao5hgny1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  136379127626064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379127625488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379127627024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379127627408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379127622416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379127625872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379127627600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379127626640: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1729784212.429239   37314 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1729784212.429258   37314 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-10-24 19:06:52.429723: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpao5hgny1
2024-10-24 19:06:52.430151: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-24 19:06:52.430164: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpao5hgny1
2024-10-24 19:06:52.433493: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-10-24 19:06:52.434166: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-24 19:06:52.456328: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpao5hgny1
2024-10-24 19:06:52.463737: I tensorflow/cc/saved_model/loader.cc

In [9]:
#ONNX
import tf2onnx
import onnx

model.output_names = ['output']


spec =(tf.TensorSpec((None, 28, 28, 1), tf.float32, name='input'), )
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec)
onnx.save_model(onnx_model, 'mnist_saved_onnx.onnx')

I0000 00:00:1729784212.662951   37314 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729784212.663932   37314 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-10-24 19:06:52.664002: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
I0000 00:00:1729784212.664307   37314 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-24 19:06:52.665222: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned

In [10]:
#size of output models
import os


def get_file_size(file_name, file_path):
    file_size = os.path.getsize(file_path)
    file_size = file_size / (1024 * 1024)
    print(f"{file_name} size: {file_size:.2f} MB")

In [11]:
get_file_size('h5 format', 'mnist_saved_h5.h5')
get_file_size('keras format', 'mnist_saved_keras.keras')
get_file_size('onnx format', 'mnist_saved_onnx.onnx')
get_file_size('tflite format', 'mnist_saved_tflite.tflite')

h5 format size: 1.43 MB
keras format size: 1.43 MB
onnx format size: 0.47 MB
tflite format size: 0.47 MB


In [21]:
#inference_time : h5
import time
from PIL import Image


model = tf.keras.models.load_model('mnist_saved_h5.h5')
#model.summary()


def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))

    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0

    img_array = img_array.reshape(1, 28, 28, 1)

    return img_array

image_path = 'three.jpg'

start_time = time.time()

test_image = preprocess_image(image_path)

predictions = model.predict(test_image)
predicted_class = np.argmax(predictions, axis=1)

end_time = time.time()

h5_inference_time = end_time - start_time

print( 'predictied_class: ', predicted_class[0])
print("h5_inference_time", h5_inference_time)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
predictied_class:  3
h5_inference_time 0.0717313289642334


In [25]:
#tflite inference time

tflite_model_path = 'mnist_saved_tflite.tflite'


interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


image_path = 'three.jpg'


test_image = preprocess_image(image_path)

start_time = time.time()

interpreter.set_tensor(input_details[0]['index'], test_image)

interpreter.invoke()

predictions = interpreter.get_tensor(output_details[0]['index'])

end_time = time.time()

predicted_class = np.argmax(predictions, axis=1)
tflite_inference_time = end_time - start_time

print( 'predictied_class: ', predicted_class[0])
print("tflite_inference_time", tflite_inference_time)



predictied_class:  3
tflite_inference_time 0.0005831718444824219


In [36]:
import onnx
import onnxruntime as ort

onnx_model_path = 'mnist_saved_onnx.onnx'
onnx_model = onnx.load(onnx_model_path)


#input_shape = onnx_model.graph.input[0].type.tensor_type.shape.dim
#print(input_shape)


session = ort.InferenceSession(onnx_model_path)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


img = 'three.jpg'

start_time = time.time()

test_image = preprocess_image(img)

predictions = session.run([output_name], {input_name : test_image})

end_time = time.time()
predicted_class = np.argmax(predictions[0], axis=1)

onnx_inference_time = end_time - start_time

print( 'predictied_class: ', predicted_class[0])
print("onnx_inference_time", onnx_inference_time)







predictied_class:  3
onnx_inference_time 0.0019366741180419922


In [38]:
print("h5_inference_time", h5_inference_time, 'seconds')
print("onnx_inference_time", onnx_inference_time, 'seconds')
print("tflite_inference_time", tflite_inference_time , 'seconds')

h5_inference_time 0.0717313289642334 seconds
onnx_inference_time 0.0019366741180419922 seconds
tflite_inference_time 0.0005831718444824219 seconds


In [ ]:
#Accuracy